In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers datasets scikit-learn
!pip install --upgrade torch torchvision

  Using cached torch-2.7.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached torchvision-0.22.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.wh

In [15]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments,EarlyStoppingCallback
from datasets import Dataset
import os
os.environ["WANDB_DISABLED"] = "true"

In [21]:
train_df=pd.read_excel("/content/train_nor_811.xlsx")
train_df_1=pd.read_csv("/content/train_backtranslate.csv")
train_df_2=pd.read_csv("/content/train_resample.csv")
valid_df=pd.read_excel("/content/valid_nor_811.xlsx")
test_df=pd.read_excel("/content/test_nor_811.xlsx")
train_df.drop(columns=["Unnamed: 0"],inplace=True)
train_df_1.drop(columns=["Unnamed: 0"],inplace=True)
train_df_2.drop(columns=["Unnamed: 0"],inplace=True)

In [6]:
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.5 MB/s eta 0:00:00


In [7]:
from underthesea import word_tokenize
def word_segment(text):
    return word_tokenize(text, format="text").replace(" ","_")

In [8]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [29]:
label2id = {
    'Anger': 0,
    'Disgust': 1,
    'Fear': 2,
    'Enjoyment': 3,
    'Sadness': 4,
    'Surprise': 5,
    'Other': 6  # nếu bạn có nhãn này
}
for df in [train_df,train_df_1,train_df_2, valid_df, test_df]:
    df['labels'] = df['Emotion'].map(label2id)

In [30]:
train_dataset = Dataset.from_pandas(train_df)
train_dataset_1=Dataset.from_pandas(train_df_1)
train_dataset_2=Dataset.from_pandas(train_df_2)
valid_dataset = Dataset.from_pandas(valid_df)
def preprocess_function(examples):
    return tokenizer(
        examples["Sentence"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_dataset = train_dataset.map(preprocess_function, batched=True)
train_dataset_1=train_dataset_1.map(preprocess_function, batched=True)
train_dataset_2=train_dataset_2.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5548 [00:00<?, ? examples/s]

Map:   0%|          | 0/8401 [00:00<?, ? examples/s]

Map:   0%|          | 0/6748 [00:00<?, ? examples/s]

Map:   0%|          | 0/686 [00:00<?, ? examples/s]

In [9]:
model=AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base",num_labels=7)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
!pip install --upgrade transformers

In [31]:
training_args=TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
    logging_dir="./logs",
    logging_steps=50,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [32]:
def compute_metrics(pred):
    logits,labels=pred
    preds=np.argmax(logits,axis=-1)
    acc=accuracy_score(labels,preds)
    return {"accuracy":acc}

In [33]:
early_stops=EarlyStoppingCallback(early_stopping_patience=3)
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stops]
    )

/tmp/ipython-input-33-945513562.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


In [34]:
train_results=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.168500,1.173743,0.583090
2,0.897300,1.117555,0.603499
3,0.640100,1.167754,0.613703
4,0.411200,1.311951,0.625364
5,0.393800,1.548671,0.626822
6,0.231500,1.973698,0.610787
7,0.254200,2.101465,0.610787
8,0.133600,2.349260,0.619534


In [35]:
trainer.save_model("./best_phobert_model")
tokenizer.save_pretrained("./best_phobert_model")
best_acc=trainer.state.best_metric
print(f"Best Accuracy: {best_acc}")

Best Accuracy: 0.6268221574344023


In [37]:
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(preprocess_function, batched=True)


test_result=trainer.predict(test_dataset)
pred_labels=np.argmax(test_result.predictions,axis=1)
true_labels=test_result.label_ids
print(classification_report(true_labels,pred_labels))


Map:   0%|          | 0/693 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

           0       0.43      0.23      0.30        40
           1       0.53      0.65      0.59       132
           2       0.65      0.65      0.65        46
           3       0.67      0.72      0.69       193
           4       0.63      0.70      0.66       116
           5       0.63      0.70      0.67        37
           6       0.63      0.42      0.50       129

    accuracy                           0.61       693
   macro avg       0.60      0.58      0.58       693
weighted avg       0.61      0.61      0.60       693



In [38]:
early_stops=EarlyStoppingCallback(early_stopping_patience=3)
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_1,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stops]
    )

/tmp/ipython-input-38-2840266532.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


In [39]:
train_results_1=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.581800,1.326362,0.622449
2,0.328800,1.992706,0.612245
3,0.191700,2.421974,0.618076
4,0.067200,2.904735,0.578717


In [40]:
trainer.save_model("./best_phobert_model_1")
tokenizer.save_pretrained("./best_phobert_model_1")
best_acc=trainer.state.best_metric
print(f"Best Accuracy: {best_acc}")

Best Accuracy: 0.6224489795918368


In [41]:
early_stops=EarlyStoppingCallback(early_stopping_patience=3)
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_2,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stops]
    )

/tmp/ipython-input-41-1961871143.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


In [ ]:
train_results_2=trainer.train()

Epoch,Training Loss,Validation Loss


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/best_phobert_model_1'